In [ ]:
#| default_exp data_generation

In [ ]:
#| export

import numpy as np
import pandas as pd
from typing import Union
from scipy.interpolate import interp1d
import tskit
from tqdm import tqdm
import msprime

from torch_geometric.utils.convert import from_networkx
import networkx as nx
import torch

import random
from scipy.special import beta
import networkx as nx
from torch_geometric.loader import DataLoader
from torch_geometric.utils.to_dense_adj import to_dense_adj
from torch_geometric.utils.to_dense_batch import to_dense_batch

In [ ]:
#| export 

def seed_everything(seed=42):
    """Seed all random number generators."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    # Ensure deterministic behavior (this can slow down training, so you might not always want it)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [ ]:
#| export

def sample_constant_population_size(n_min:int=10, n_max:int=100_000, num_time_windows=21):
    return np.random.uniform(n_min, n_max, 1).tolist() * num_time_windows

In [ ]:
#| export

def sample_population_size(n_min:int=10, n_max:int=100_000, num_time_windows=21):
    
    """Creates random demography. Function taken from: 
    https://gitlab.inria.fr/ml_genetics/public/dlpopsize_paper
    
    :param int n_min: Lower-bound of demography.
    :param int n_max: Upper-bound of demography.
    :param int num_time_windows: Number of population sizes in demography.
    :return list: 
    """
    
    n_min_log10 = np.log10(n_min)
    n_max_log10 = np.log10(n_max)
    population_size = [10 ** np.random.uniform(low=n_min_log10, high=n_max_log10)] 
    for j in range(num_time_windows - 1):
        population_size.append(10 ** n_min_log10 - 1)
        while population_size[-1] > 10 ** n_max_log10 or population_size[-1]  < 10 ** n_min_log10:
            population_size[-1] = population_size[-2] * 10 ** np.random.uniform(-1, 1)
            
    return population_size

In [ ]:
#| export
def get_population_time(time_rate:float=0.06, tmax:int = 130_000,
                        num_time_windows:int = 21
                       ) -> np.array :
    """Creates population time points; used as time points to change
    population size changes for simulation
    
    :return numpy.ndarray: time points of length num_time_windows
    """
    
    population_time = np.repeat([(np.exp(np.log(1 + time_rate * tmax) * i /
                              (num_time_windows - 1)) - 1) / time_rate for i in
                              range(num_time_windows)], 1, axis=0)
    population_time[0] = 1
    return population_time

In [ ]:
#| export
def simulate_scenario(population_size: Union[list, np.ndarray],
                      population_time: Union[list, np.ndarray],
                      mutation_rate: float,
                      recombination_rate: float,
                      segment_length: float,
                      num_sample:int,
                      num_replicates: int,
                      seed: int = 69420,
                      model = None,
                     ):

    """ Simulates tree sequence with msprime given population size changes at specific time-points.
    Piece-wise constant simualtion of demography.
    
    :return: generator of tskit.trees.TreeSequence
    """

    demography=msprime.Demography()
    demography.add_population(initial_size=(population_size[0]))
    for i, (time, size) in enumerate(zip(population_time, population_size)):
        if i != 0:
            demography.add_population_parameters_change(time=time, initial_size=size)

    tss = msprime.sim_ancestry(samples=num_sample, recombination_rate=recombination_rate,
                                          sequence_length=int(segment_length), demography=demography,
                                          ploidy=1, model=model, num_replicates=num_replicates, random_seed=seed)

    return tss

In [ ]:
#| export
def sample_parameters(num_time_windows = 60,
                      n_min = 10_000,
                      n_max = 10_000_000,
                      recombination_rates = [1e-8, 1e-8],
                      population_size= None,
                      model = None,
                      
                      ) -> pd.DataFrame:
    

    parameter_names = ["recombination_rate"]
    for i in range(num_time_windows): parameter_names.append("pop_size_" + str(i))
    parameter_names.append("model")
    parameters = []
    recombination_rate = np.random.uniform(low=recombination_rates[0], high=recombination_rates[1])

    if population_size is None:
        population_size = sample_population_size(n_min=n_min, n_max=n_max, num_time_windows=num_time_windows)
    
    parameter = [recombination_rate]
    for current_population_size in population_size: 
        parameter.append(current_population_size)
    parameter.append(model)
    parameters.append( parameter )
    parameters = pd.DataFrame(parameters, columns=parameter_names)
    
    return parameters

In [ ]:
#| export

def sample_population_size(n_min:int=10, n_max:int=100_000, num_time_windows=21):
    
    """Creates random demography. Function taken from: 
    https://gitlab.inria.fr/ml_genetics/public/dlpopsize_paper
    
    :param int n_min: Lower-bound of demography.
    :param int n_max: Upper-bound of demography.
    :param int num_time_windows: Number of population sizes in demography.
    :return list: 
    """
    
    n_min_log10 = np.log10(n_min)
    n_max_log10 = np.log10(n_max)
    population_size = [10 ** np.random.uniform(low=n_min_log10, high=n_max_log10)] 
    for j in range(num_time_windows - 1):
        population_size.append(10 ** n_min_log10 - 1)
        while population_size[-1] > 10 ** n_max_log10 or population_size[-1]  < 10 ** n_min_log10:
            population_size[-1] = population_size[-2] * 10 ** np.random.uniform(-1, 1)
            
    return population_size

In [ ]:
#| export
def sample_smooth_population_parameters():

    upper_out_of_bound = lower_out_of_bound = True
    while upper_out_of_bound or lower_out_of_bound:
        steps = 18
        x = np.log(get_population_time(time_rate=0.1, num_time_windows=steps, tmax=10_000_000).tolist())
        y = np.log(sample_population_size(10_000, 10_000_000, steps))
        xnew = np.linspace(x[0], x[-1], num=10000, endpoint=True)
        f_cubic = interp1d(x, y, kind='cubic')
        ynew = f_cubic(xnew)
        upper_out_of_bound = np.sum(np.exp(ynew) > 10_000_000) > 0
        lower_out_of_bound = np.sum(np.exp(ynew) < 10_000) > 0
        x_sample = xnew[np.linspace(10, 9999, 60).astype(int)]
        y_sample = ynew[np.linspace(10, 9999, 60).astype(int)]
        population_time = np.exp(x_sample)
        population_size = np.exp(y_sample)
        
    return population_time, population_size

In [ ]:
#| export
def simulate_tree_sequence(parameters: pd.DataFrame,
                           population_time: list, 
                           segment_length = 1e6, 
                           num_sample = 10, 
                           num_replicates = 100,
                           seed = 69420,
                          ):
    
    tree_sequences = []
    population_size = parameters.loc["pop_size_0":"pop_size_" + str(len(population_time)-1)].tolist()
    recombination_rate = parameters.loc["recombination_rate"]
    model = parameters.loc["model"]
    
    if type(model) == np.float64:
        model = msprime.BetaCoalescent(alpha=model)
    else:
        model = None
        
    tree_sequences = simulate_scenario(population_size=population_size,
                    population_time=population_time,
                    mutation_rate=0, # otherwise memory not sufficient
                    recombination_rate=recombination_rate,
                    segment_length=segment_length,
                    num_sample=num_sample,
                    num_replicates=num_replicates,
                    seed=seed, model=model)
        
    tss = []
    for ts in tree_sequences:
        tss.append(ts)    
        
    return tss

In [ ]:
#| export
def generate_sample(nth_scenario, num_sim_trees, alpha):
    
    sequence_length = 1_000_000
    #alpha = np.round(np.random.uniform(1.01, 1.99), 2)
    population_time , population_size = sample_smooth_population_parameters()
   
    parameter_set = sample_parameters(
       num_time_windows=60,
       n_min = 10_000,
       n_max = 10_000_000,
       recombination_rates=[1e-8, 1e-8],
       population_size=population_size,
       model=alpha   
    )

    num_replicates = 1
    ts = simulate_tree_sequence(
        parameter_set.iloc[0],
        population_time=population_time,
        segment_length=sequence_length,
        num_replicates=num_replicates, 
        seed=nth_scenario+1, #+1*1000,
    )[0]

    while ts.num_trees < num_sim_trees:
        sequence_length = sequence_length * 1.2
        ts = simulate_tree_sequence(
            parameter_set.iloc[0],
            population_time=population_time,
            segment_length=sequence_length,
            num_replicates=num_replicates, 
            seed=nth_scenario+1,
        )[0]

    return ts, population_size, alpha

In [ ]:
#| export

# def alternative_coalescent_mask(ts, population_time, x_times_std=2, n_trees=500):
    
#     trees = ts.aslist()[0:n_trees]
#     nodes_n_trees = []
#     for tree in trees:
#         nodes_n_trees += list(tree.nodes())
    
#     node_times = [ts.get_time(node.id) for node in ts.nodes() if node.id >= ts.num_samples and node.id in nodes_n_trees]
    
#     log_node_times = np.log(node_times)
#     mean = log_node_times.mean()
#     std = log_node_times.std()
#     lowerbound = np.exp(mean-x_times_std*std)
#     upperbound = np.exp(mean+x_times_std*std)
#     mask1 = population_time > lowerbound
#     mask2 = population_time < upperbound
#     mask = np.logical_and(mask1, mask2)
#     return mask


def alternative_coalescent_mask(ts, population_time, threshold=10, num_sim_trees=500, nth_tree=1):
    
    #trees = ts.aslist()[0:num_sim_trees:nth_tree]
    nodes_n_trees = []
    #for tree in trees: 
    #    nodes_n_trees += list(tree.nodes())

    for i, tree in enumerate(ts.trees()):
        if i < num_sim_trees and i % nth_tree == 0:
            nodes_n_trees += list(tree.nodes())
    
        
    node_times = [ts.get_time(node.id) for node in ts.nodes() if node.id >= ts.num_samples and node.id in nodes_n_trees]
    hist, bin_edges = np.histogram(node_times, bins=population_time)

    mask = np.zeros(60) != 0
    mask[1:] = hist > threshold
    
    start = np.argwhere(mask == 1)[0]
    end = np.argwhere(mask == 1)[-1]
    mask[int(start):int(end+1)] = True
    return mask

9

In [ ]:
#| export

def ts_to_data_objects(ts, num_sim_trees=500, nth_tree=1, num_embedding=60, y=None):
    max_num_nodes = 2 * ts.num_samples - 1
    data_objects = []
    
    #trees = ts.aslist()[0:num_sim_trees:nth_tree]
    for i, tree in enumerate(ts.trees()):
        if i < num_sim_trees and i % nth_tree == 0:
            
            data = from_networkx(nx.Graph(tree.as_dict_of_dicts()))
            data.edge_weight = data.branch_length 
            data.branch_length  = None
            data.x = torch.eye(max_num_nodes, num_embedding)
            data.x[data.num_nodes:] = torch.zeros(num_embedding)
            data.num_nodes = max_num_nodes
            data.y = y
            data_objects.append(data)
    return data_objects

In [ ]:
#| export 

def calculate_beta_coal_ne_estimate(theta, sample_size, L, alpha, mu_real):
    mu_estimated = theta / ((2*np.sum(1/np.array(range(1, sample_size)))) * L)
    m = 1 + (1/((2**(alpha-1))*(alpha-1)))
    scale = (m**alpha)/(alpha * beta(2-alpha,alpha))
    Ne = (((mu_estimated/mu_real)/scale)**(1/(alpha-1)))
    return Ne

In [ ]:
#| export

def generate_batch(
    seed, alpha, batch_size, num_trees, num_sim_trees, nth_tree=1, scaling="log", mutation_rate = 4e-8):

    tss, demographies, alphas = zip(*[generate_sample(seed+i, num_sim_trees, alpha) for i in range(batch_size)])
    demographies = np.array(demographies)


    # based on num_sim_trees
    tss = [msprime.mutate(ts, rate=mutation_rate) for ts in tss]
    ne_norm_factors = [calculate_beta_coal_ne_estimate(ts.num_mutations,
                                                     ts.sample_size, ts.sequence_length,
                                                      alphas[i], mutation_rate) for i, ts in enumerate(tss)]
    #ne_norm_factors = demographies.mean(1)
        
    population_time, _ = sample_smooth_population_parameters()
    masks = [alternative_coalescent_mask(ts, population_time, num_sim_trees=num_sim_trees, threshold=10, nth_tree=nth_tree) for ts in tss]
    
    data_objects = []
    for i, ts in enumerate(tss):
        ne_norm_factor = ne_norm_factors[i]
        ts_data_objects = ts_to_data_objects(ts, num_sim_trees=num_sim_trees,nth_tree=nth_tree, num_embedding=60)

        
        if scaling=="log": demographies[i] = np.log(demographies[i])
        if scaling=="log_ne": demographies[i] = np.log(demographies[i] / ne_norm_factor)
        if scaling=="ne": demographies[i] = demographies[i] / ne_norm_factor

        
        for data_object in ts_data_objects: 
            if scaling=="log": data_object.edge_weight = np.log(data_object.edge_weight)
            if scaling=="log_ne": data_object.edge_weight = np.log(data_object.edge_weight  / ne_norm_factor)
            if scaling=="ne": data_object.edge_weight = data_object.edge_weight  / ne_norm_factor
                
        data_objects += ts_data_objects    
    
    dl = DataLoader(data_objects, batch_size=num_trees*batch_size)

    
    batch = next(iter(dl))
    adj_batch = to_dense_adj(batch.edge_index, batch=batch.batch,
                        edge_attr=batch.edge_weight, max_num_nodes=None)
    
    x_batch, _ = to_dense_batch(batch.x, batch.batch)

    masks_batch = torch.Tensor(np.array(masks))
    alpha_batch = torch.Tensor(alphas)
    demography_batch = torch.Tensor(demographies)
    ne_norm_factors = torch.Tensor(ne_norm_factors)
    
    return x_batch, adj_batch, masks_batch, demography_batch, alpha_batch, ne_norm_factors

In [ ]:
#| export 

def continously_add_batch_to_queue(
    data_queue,
    process_id,
    num_processes,
    seed_start_value,
    num_trees = 1000,
    num_sim_trees = 1000,
    nth_tree = 1,
    batch_size = 4,
    scaling="log",
    mutation_rate = 4e-8
):
    
    while True:
        seed_value = process_id + seed_start_value
        device = None
        alpha = np.round(np.random.uniform(1.01, 1.99), 2)
        try:
            x_batch, adj_batch, masks_batch, demography_batch, alpha_batch, ne_norm_factors = generate_batch(
                seed_value, alpha, batch_size=batch_size, num_trees=num_trees,num_sim_trees=num_sim_trees,
                nth_tree=nth_tree, scaling=scaling, mutation_rate=mutation_rate)
            data_point = [x_batch, adj_batch, masks_batch, demography_batch, alpha_batch]
            data_queue.put(data_point)
            seed_value += num_processes
        except Exception as e:
            print(e, seed_value, alpha)



In [ ]:
#| export

def get_next_batch(data_queue): return data_queue.get()

In [ ]:
#np.random.seed(0x1337)

In [ ]:
# tss, demographies, alphas = [], [], []
# for i in range(1):
#     ts, demography, alpha = generate_sample(i)
#     tss.append(ts)
#     demographies.append(demography)
#     alphas.append(alpha)
# print(alphas)

In [ ]:
#population_time, _ = sample_smooth_population_parameters()

In [ ]:
#alternative_coalescent_mask(ts, population_time)